In [1]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
import json
import pickle
import string
from DiacriticDataset import DiacriticDataset

In [2]:
train_dataset = DiacriticDataset('Dataset/train_cleaned_noshadda_215.txt','Dataset/letter_to_id.pickle','Dataset/id_to_letter.pickle','Dataset/diacritic_to_id.pickle','Dataset/id_to_diacritic.pickle','Dataset/word_to_id.pickle','Dataset/id_to_word.pickle')

In [3]:
train_dataloader = DataLoader(train_dataset)

In [52]:
class AutoDiacriticNet(nn.Module):
    def __init__(self,word_emb_dim, hidden_dim, letter_vocab_size,diacritic_vocab_size,word_vocab_size):
        super(AutoDiacriticNet, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.letter_embedding = nn.Embedding(letter_vocab_size, word_emb_dim)
        self.lstm = nn.LSTM(input_size=word_emb_dim,hidden_size=hidden_dim,num_layers=2,bidirectional=True,batch_first=True)
        self.linear1 = nn.Linear(in_features=2*hidden_dim,out_features=2*hidden_dim)
        self.linear2 = nn.Linear(in_features=2*hidden_dim,out_features=2*hidden_dim)
        self.output = nn.Linear(in_features=2*hidden_dim,out_features=diacritic_vocab_size)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax()
        
    def forward(self,letters,words):
        model_input = letters.to(self.device)
        embedded = self.letter_embedding(model_input)
        lstm_out, _ = self.lstm(embedded)
        linear1_out = self.relu(self.linear1(lstm_out.view(letters.size()[1],-1)))
        linear2_out = self.relu(self.linear2(linear1_out))
        output = self.softmax(self.output(linear2_out))
        return output
        

In [96]:
EPOCHS = 50
LETTER_EMBEDDING_DIM = 5
HIDDEN_DIM = 256

letter_vocab_size = len(train_dataset.letter_to_id)
diacritic_vocab_size = len(train_dataset.diacritic_to_id)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

model = AutoDiacriticNet(LETTER_EMBEDDING_DIM,HIDDEN_DIM,letter_vocab_size,diacritic_vocab_size,None)

if use_cuda:
    model.cuda()
    
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
acumulate_grad_steps = 1

In [97]:
loss_list_train = []
for epoch in range(EPOCHS):
    loss_train_total = 0
    i = 0
    for batch_idx, input_data in enumerate(train_dataloader):
        if batch_idx == 10:
            break
            
        i += 1
        letters = input_data[0]
        diacritics = input_data[1]
        labels = diacritics[0].to(model.device)
        probs = model(letters,None)
        loss = criterion(probs,labels)
        loss = loss/ acumulate_grad_steps
        loss.backward()
        
        if i % acumulate_grad_steps == 0:
            optimizer.step()
            model.zero_grad()
        
        loss_train_total += loss.item()
            
    
    loss_train_total = loss_train_total / len(train_dataset)
    loss_list_train.append(float(loss_train_total))
    e_interval = i
    print("Epoch {} Completed,\tTrain Loss: {}".format(epoch + 1, np.mean(loss_list_train[-e_interval:])))

/home/student/.conda/envs/project/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 Completed,	Train Loss: 0.0006028543260481558
Epoch 2 Completed,	Train Loss: 0.0005832081970371062
Epoch 3 Completed,	Train Loss: 0.0005690375213703564
Epoch 4 Completed,	Train Loss: 0.0005607331110565095
Epoch 5 Completed,	Train Loss: 0.0005548444636856569
Epoch 6 Completed,	Train Loss: 0.0005503906108371068
Epoch 7 Completed,	Train Loss: 0.0005456376579244291
Epoch 8 Completed,	Train Loss: 0.0005363645537998115
Epoch 9 Completed,	Train Loss: 0.0005199899416014622
Epoch 10 Completed,	Train Loss: 0.000504721778217103
Epoch 11 Completed,	Train Loss: 0.000479592698602962
Epoch 12 Completed,	Train Loss: 0.00045699893084344444
Epoch 13 Completed,	Train Loss: 0.00043398318226936516
Epoch 14 Completed,	Train Loss: 0.0004087689459316828
Epoch 15 Completed,	Train Loss: 0.0003827454257482663
Epoch 16 Completed,	Train Loss: 0.00035563471321100763
Epoch 17 Completed,	Train Loss: 0.00032751108114642357
Epoch 18 Completed,	Train Loss: 0.00030173312271681186
Epoch 19 Completed,	Train Loss: 0.

In [100]:
true_prediction = 0
total = 0
for batch_idx, input_data in enumerate(train_dataloader):
    if batch_idx == 10:
        break
        
    letters = input_data[0]
    diacritics = input_data[1]
    labels = diacritics[0].to(model.device)
    probs = model(letters,None)
    _, predicted = torch.max(probs.data, 1)
    true_prediction += ((predicted==labels).sum()).item()
    total += letters.size()[1]
print("Accuarcy: ", true_prediction/total)


Accuarcy:  1.0


/home/student/.conda/envs/project/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
